In [ ]:
1. init data
2. train vae
3. use vae as wrapper for env
4. make other wrappers, prepro, frame stack, FRAME CAPTURE
5. train rl, gather ne imgs with capture wrapper ( also capture actions for rl)
6. if n of new img == num of old imgs
    Retrain vae
    else continue rl training and img gathering
7. continou train vae
8. performance check of rl:
    if better or == to previous performance
    continou training that rl and collecting imgs with it
    else train completle new rl as often until its better than previous.. 
    if it doesnt work
    


In [5]:
import time
import os
import torch
from torchvision import transforms
import random
import numpy as np
import yaml
from PIL import Image
from experiment import VAEXperiment
from models import *


import gymnasium as gym
from gymnasium import ObservationWrapper
from gymnasium.wrappers import PixelObservationWrapper, FrameStack
from gymnasium.spaces import Box, Discrete



from stable_baselines3 import SAC, PPO, A2C
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.callbacks import CallbackList, CheckpointCallback, EvalCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy

In [6]:
from gymnasium import ObservationWrapper
from gymnasium.wrappers import PixelObservationWrapper, FrameStack
from gymnasium.spaces import Box, Discrete
import numpy as np
import yaml
from experiment import VAEXperiment
from models import *
import torch
from torchvision import transforms
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv

class AddGaussianNoise(object):
    def __init__(self, mean=0., std=0.1):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

#ideas
#maybe use wrapper to catch and save frames during RL model learning
#alternative is to use the model afterwards to generate frames during a test run
#
#??return mu, std or sample or just mu??
#
class VAE_ENC(ObservationWrapper):
    def __init__(self, env, vae, latent_dim,
                 mean=0,std=0.1,
                 size=(64,64),
                 collect_frames_dir = None,
                 start_index = 0):
        super().__init__(env)
        #new obs space with std
        #self.observation_space = Box(shape=(2, latent_dim), low=-np.inf, high=np.inf)
        #just mean
        self.observation_space = Box(shape=(latent_dim,), low=-np.inf, high=np.inf)
        
        self.vae = vae
        #transforms
        self.mean = mean
        self.std = std
        self.size = size
        
        self.collect_frames_dir = collect_frames_dir
        self.frame_idx = start_index
        
        
        
    def observation(self, obs):
        #get frame
        #print(obs)
        frame = obs['pixels']#.to('cuda')
        if self.collect_frames_dir == None:
            im = Image.fromarray(np.array(frame))
            im.save(self.collect_frames_dir+'_'+str(self.frame_idx)+'.jpeg')
            self.frame_idx += 1
        #transform for VAE
        val_transforms = transforms.Compose([transforms.ToTensor(),
        #transforms.RandomHorizontalFlip(),
        AddGaussianNoise(self.mean, self.std),
        transforms.Resize(self.size),
        #transforms.Grayscale(),
        #transforms.Normalize(self.mean, self.std),
        ])
        frame = val_transforms(frame) #(c,h,w)
        frame = torch.unsqueeze(frame, 0)#.to(self.device) #make it (1,c,h,w)
        enc = self.vae.encode(frame)    
        enc = np.array([tensor.detach().cpu().numpy() for tensor in enc])
        #with std
        #enc = np.array([enc[0][0], enc[1][0]]) ## mu, std #  give only mu?
        #just mean
        enc = np.array(enc[0][0])
        

            
        
        
        return enc
    
def get_vae(version='version_0',log_directory='logs/BCE_sum_VAE/MSSIMVAE/'):

    model_path=log_directory+'/'+version+'/hparams.yaml'
    ckpt_path=log_directory+'/'+version+'/checkpoints/last.ckpt'

    config = yaml.safe_load(open(model_path))
    model = vae_models[config['model_params']['name']](**config['model_params'])
    ckpt = torch.load(ckpt_path)
    experiment = VAEXperiment(model, config['exp_params'])
    experiment.load_state_dict(ckpt['state_dict'])      
    vae = experiment.model
    return vae

In [ ]:
data_name='wrapper_test'
save_path='Data/MountainCar/'+data_name+'/'

vae = get_vae(version='version_13',log_directory='logs/MountainCar/BCE_sum_VAE_2/MSSIMVAE/')

In [ ]:
env = gym.make("MountainCarContinuous-v0",
                render_mode ='rgb_array')
seed = 42
env.reset(seed=seed)
env = PixelObservationWrapper(env)
env = VAE_ENC(env, vae, 2, collect_frames_dir = save_path)
env = FrameStack(env, num_stack=2)
env = Monitor(env)

In [ ]:
env.reset()

In [3]:
class frame_saver(ObservationWrapper):
    def __init__(self, env,
                 collect_frames_dir = None,
                 start_index = 0):
        super().__init__(env)
        
        self.collect_frames_dir = collect_frames_dir
        self.frame_idx = start_index
                
        
    def observation(self, obs):
        frame = obs['pixels']#.to('cuda')
        if self.collect_frames_dir != None:
            im = Image.fromarray(np.array(frame))
            im.save(self.collect_frames_dir+'_'+str(self.frame_idx)+'.jpeg')
            self.frame_idx += 1
        return obs

In [4]:

def make_env(env_id: str = "MountainCarContinuous-v0", rank: int = 0, seed: int = 42) -> Callable:
    def _init() -> gym.Env:
        data_name='wrapper_test'
        save_path='Data/MountainCar/'+data_name+'/real_eval_env_'+str(rank)+'_'
        vae = get_vae(version='version_13',log_directory='logs/MountainCar/BCE_sum_VAE_2/MSSIMVAE/')
        
        env = gym.make(env_id,
                    render_mode ='rgb_array')
        seed = 42
        env.reset(seed=seed + rank)
        env = PixelObservationWrapper(env)
        env = frame_saver(env, save_path)
        env = VAE_ENC(env, vae, 2, collect_frames_dir = save_path)
        env = FrameStack(env, num_stack=2)
        env = Monitor(env)
        return env
    set_random_seed(seed)
    return _init

In [9]:
env_id = "MountainCarContinuous-v0"
num_cpu = 2  # Number of processes to use
# Create the vectorized environment
env = SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)])

In [10]:
# Tuned
agent = A2C(

    env = env,
    n_steps= 100,
    
    policy='MlpPolicy',
    ent_coef= 0.0,
    use_sde=True,
    sde_sample_freq = 16,
    policy_kwargs= dict(log_std_init=0.0, ortho_init=False))

In [11]:
RL_train_steps = 10
agent.learn(total_timesteps=RL_train_steps)

In [ ]:
env_id = "MountainCarContinuous-v0"
num_cpu = 2 
eval_env = SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)])

In [ ]:
mean_reward, std_reward = evaluate_policy(agent, eval_env, n_eval_episodes=2)